# Распознавание образов, описываемых гауссовскими случайными векторами с одинаковыми матрицами ковариаций

### 1. Задание исходных данных

In [306]:
import numpy as np

In [250]:
m = np.matrix([[-5,1,0],[1,-4,8],[10,-2,1]]) #матожидания
C = np.matrix([[3, 1, 1],[1, 3, 1],[ 1, 1, 3]]) #матрица ковариаций

In [251]:
n = 3 #размерность признакового пространства
M = 3 #число классов

In [252]:
pw = [0.4,0.3,0.3] #апостериорные вероятности

In [333]:
import math
m=np.matrix([[0, 4],[0, 1]])
n = 2
M = 2
pw = [0.9,0.1]

C=np.zeros((n,n))
D=1
ro = 0.7
alf  = - math.log(ro)
for i in range(n):
    for j in range(n):
        C[i,j] = D*math.exp(-alf*abs(i-j))
C = np.matrix(C)

In [336]:
C

matrix([[ 1. ,  0.7],
        [ 0.7,  1. ]])

### 2. Расчет разделяющих функций и вероятностей ошибок разпознавания

In [334]:
G = np.zeros((M,n+1))
P_err, l0 = np.zeros((M,M)), np.zeros((M,M))

In [335]:
C_ = np.invert(C)

TypeError: ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [311]:
from scipy.stats import norm

In [312]:
for i in range(M):
    G[i,range(n)] = np.transpose(C_*m[:,0])
    G[i,n] = -0.5*np.transpose(m[:,i])*C_*m[:,i]
    for j in range(i+1,M):
        l0[i,j] = np.log(pw[j]/pw[i])
        h = 0.5*np.transpose((m[:,i]-m[:,j]))*C_*(m[:,i]-m[:,j])
        sD = np.sqrt(np.abs(h))
        P_err[i,j] = norm.cdf(l0[i,j],h,sD)
        P_err[j,i] = 1 - norm.cdf(l0[i,j],-h,sD)
    P_err[i,i] = 1 - np.sum(P_err[i,:])
        

In [313]:
print(P_err)

[[  7.82701129e-04   9.99217299e-01]
 [  9.99217299e-01   7.82701129e-04]]


### 3. Тестирование алгоритма методом статистических испытаний

In [314]:
from numpy.linalg import cholesky
def randncor(n,N,C):
    try:
        A = cholesky(C)
    except LinAlgError:
        m=0
        print('A is not positive definite')
    m = n
    u = np.random.randn(m,N)
    x = np.transpose(A)*u
    return x     

In [315]:
x = np.matrix(np.ones((n+1,1)))
P_err_experiment = np.zeros((M,M))
K = 10000

In [316]:
for k in range(K):
    for i in range(M):
        x_ = randncor(n,1,C)
        x[0:n,0] = m[:,i] + x_
        u = G*x+np.log(np.transpose(pw))
        um,umi = u.max(), np.where(u == u.max())[1]
        P_err_experiment[i,umi] = P_err_experiment[i,umi]+1

In [317]:
P_err_experiment /=K 

In [318]:
P_err_experiment

array([[ 1.,  1.],
       [ 1.,  1.]])